# LMing-Rules baseline
* Using language modeling.
* Using rules, extracted from the training data.

### Sign in
* To get the data.

In [1]:
%%capture
!pip install aicrowd-cli
%load_ext aicrowd.magic

In [2]:
%aicrowd login

Please login here: https://api.aicrowd.com/auth/z7jfi7lD9Z1E-ckzz5WqxFjt8VNDGAPwMyk-D1fC-5Q
API Key valid
Gitlab access token valid
Saved details successfully!


In [3]:
!rm -rf data
!mkdir data
%aicrowd ds dl -c htrec-2022 -o data

official_ranking.csv:   0%|          | 0.00/19.4k [00:00<?, ?B/s]

original_test.csv:   0%|          | 0.00/37.2k [00:00<?, ?B/s]

synthetic_test.csv:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/45.5k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/395k [00:00<?, ?B/s]

In [4]:
%%capture
!pip install pywer
import pywer
import pandas as pd
import numpy as np
import os

In [5]:
# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
print(f"{train_df.shape[0]} train and {test_df.shape[0]} instances"); train_df.sample()

1875 train and 338 instances


,HUMAN_TRANSCRIPTION,SYSTEM_TRANSCRIPTION,CENTURY,IMAGE_PATH,TEXT_LINE_NUM
1871,γω την ημετεραν υπεδυ σαρκα και τ αλλα παν,γω την η μετεραν υπεθσαρκα ειταλλαπαν,10,100 Bodleian-Library-MS-Barocci-184_00040_fol-...,26


# Models

### B4: BERT

Bibliography:

https://arxiv.org/pdf/2002.06823.pdf

Bert2Bert:

https://colab.research.google.com/drive/1WIk2bxglElfZewOHboPFNj8H44_VAyKE?usp=sharing#scrollTo=z0FWPCFnCgf6


#### Load Libraries

In [7]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [8]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [9]:
%%capture
!pip install transformers

#### Convert DF to TF

In [10]:
# https://medium.com/when-i-work-data/converting-a-pandas-dataframe-into-a-tensorflow-dataset-752f3783c168

In [11]:
train_df[["HUMAN_TRANSCRIPTION", "SYSTEM_TRANSCRIPTION"]].head(3)

,HUMAN_TRANSCRIPTION,SYSTEM_TRANSCRIPTION
0,ἐγγινομένα πάθη μὴ σβεννύντες ἀλλὰ τῆ εκλύσει,ἐγγενομεναπαδημησμεννωτες ἀλλατῆε κλησει
1,τοῦ βίου τοῦ καθ ΄ εαυτοὺς πολλὰ γίνεσθαι συγχ...,του β ου του καλεαυτοὺς πολλαγινεσθαι συγχωρ όν
2,τες ἐμπυρίζουσι τὸν ἀμπελῶνα ἀλλὰ καὶ ὁ διὰ,τες εμπυριζου σιμαμπελῶνα ἀλλακαι ὅδξα


In [12]:
# training_dataset = (
#     tf.data.Dataset.from_tensor_slices(
#         (
#             tf.cast(train_df[['HUMAN_TRANSCRIPTION', 'SYSTEM_TRANSCRIPTION']].values, tf.string),
#         )
#     )
# )

#### Tokinizer

In [13]:
from transformers import BertTokenizerFast
# tokenizer = BertTokenizerFast.from_pretrained("pranaydeeps/Ancient-Greek-BERT")
tokenizer = BertTokenizerFast.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [14]:
MAX_LEN = 120
encoder_max_length=MAX_LEN
decoder_max_length=MAX_LEN

In [15]:
tokenizer('ἐγγινομένα πάθη μὴ σβεννύντες ἀλλὰ', padding="max_length", truncation=True,
                     max_length=encoder_max_length)

{'input_ids': [101, 2186, 13968, 4785, 6817, 452, 251, 8981, 273, 281, 5429, 374, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [16]:
tokenizer.convert_ids_to_tokens([101, 2186, 13968, 4785, 6817])

['[CLS]', 'εγ', '##γιν', '##ομενα', 'παθη']

In [17]:
tokenizer('ἐγγενομεναπαδημησμεννωτες ἀλλατῆε κλησει', padding="max_length", truncation=True,
                     max_length=encoder_max_length)

{'input_ids': [101, 2186, 5937, 4785, 858, 17903, 10247, 273, 1622, 605, 374, 564, 265, 9822, 532, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [18]:
tokenizer.convert_ids_to_tokens([101, 2186, 5937, 4785, 858, 17903, ])

['[CLS]', 'εγ', '##γεν', '##ομενα', '##πα', '##δημη']

In [19]:
def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["SYSTEM_TRANSCRIPTION"].values.tolist(), padding="max_length", truncation=True,
                     max_length=encoder_max_length)
  outputs = tokenizer(batch["HUMAN_TRANSCRIPTION"].values.tolist(), padding="max_length", truncation=True,
                      max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

In [20]:
train_data = train_df[["HUMAN_TRANSCRIPTION", "SYSTEM_TRANSCRIPTION"]]

In [21]:
train_data = process_data_to_model_inputs(train_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [22]:
train_data.head(3)

,HUMAN_TRANSCRIPTION,SYSTEM_TRANSCRIPTION,input_ids,attention_mask,decoder_input_ids,decoder_attention_mask,labels
0,ἐγγινομένα πάθη μὴ σβεννύντες ἀλλὰ τῆ εκλύσει,ἐγγενομεναπαδημησμεννωτες ἀλλατῆε κλησει,"[101, 2186, 5937, 4785, 858, 17903, 10247, 273...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2186, 13968, 4785, 6817, 452, 251, 8981,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2186, 13968, 4785, 6817, 452, 251, 8981,..."
1,τοῦ βίου τοῦ καθ ΄ εαυτοὺς πολλὰ γίνεσθαι συγχ...,του β ου του καλεαυτοὺς πολλαγινεσθαι συγχωρ όν,"[101, 346, 234, 247, 281, 346, 5608, 5999, 411...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 346, 4466, 346, 1191, 106, 11300, 500, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[101, 346, 4466, 346, 1191, 106, 11300, 500, 1..."
2,τες ἐμπυρίζουσι τὸν ἀμπελῶνα ἀλλὰ καὶ ὁ διὰ,τες εμπυριζου σιμαμπελῶνα ἀλλακαι ὅδξα,"[101, 1354, 278, 5120, 8052, 5510, 389, 22396,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1354, 278, 5120, 8052, 5510, 3810, 362, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[101, 1354, 278, 5120, 8052, 5510, 3810, 362, ..."


In [23]:
val_data = train_data.iloc[:50]
val_data.reset_index(inplace=True)

test_data = train_data.iloc[50:60]
test_data.reset_index(inplace=True)

train_data = train_data.iloc[60:1000]
train_data.reset_index(inplace=True)

In [24]:
print(val_data.shape)
print(test_data.shape)
print(train_data.shape)

(50, 8)
(10, 8)
(940, 8)


In [25]:
# train_data.drop(columns=['HUMAN_TRANSCRIPTION', 'SYSTEM_TRANSCRIPTION'],inplace=True)

In [26]:
# item = {key: torch.as_tensor(val) for key, val in train_data.items()}
# item

In [27]:
from torch.utils.data import Dataset, DataLoader

class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):

        # step 1: get the sentence and word labels 
        sentence = self.data["SYSTEM_TRANSCRIPTION"][index]#.strip().split()  
        encoded_labels = self.data["labels"][index]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        encoding = self.tokenizer(sentence, 
                                padding="max_length",
                                truncation=True,
                                max_length=self.max_len)

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}

        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [28]:
training_set = dataset(train_data, tokenizer, MAX_LEN)
validation_set = dataset(val_data, tokenizer, MAX_LEN)
test_set = dataset(test_data, tokenizer, MAX_LEN)

In [29]:
training_set[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([  101,  3258, 13881,  1987,   269,   344,   355,  6186,  4993,   404,
         10012,   265,   267,  5668,   275,   267,  8276,  6298,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
         

In [30]:
# train_data.set_format(
#     type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
# )

In [31]:
# val_data = val_data.map(
#     process_data_to_model_inputs, 
#     batched=True, 
#     batch_size=batch_size, 
#     remove_columns=["article", "highlights", "id"]
# )

In [32]:
# val_data.set_format(
#     type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
# )

#### Model


Hugging Face Bert:

https://huggingface.co/pranaydeeps/Ancient-Greek-BERT

OR
(https://huggingface.co/nlpaueb/bert-base-greek-uncased-v1)

In [33]:
from transformers import EncoderDecoderModel
# bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("pranaydeeps/Ancient-Greek-BERT", "pranaydeeps/Ancient-Greek-BERT")
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("nlpaueb/bert-base-greek-uncased-v1", "nlpaueb/bert-base-greek-uncased-v1")

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at nlpaueb/bert-base-greek-uncased-v1 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at nlpaueb/bert-base-greek-uncased-v1 were not used when initializing

In [34]:
from transformers import BertGenerationEncoder,EncoderDecoderModel,BertGenerationDecoder
# # leverage checkpoints for Bert2Bert model...
# # use BERT's cls token as BOS token and sep token as EOS token
# encoder = BertGenerationEncoder.from_pretrained("pranaydeeps/Ancient-Greek-BERT", 
#                                                 bos_token_id=101,
#                                                 eos_token_id=102)
# # add cross attention layers and use BERT's cls token as BOS token and sep token as EOS token
# decoder = BertGenerationDecoder.from_pretrained("pranaydeeps/Ancient-Greek-BERT",
#                                                 add_cross_attention=True,
#                                                 is_decoder=True, 
#                                                 bos_token_id=101, 
#                                                 eos_token_id=102)
# bert2bert = EncoderDecoderModel(encoder=encoder, decoder=decoder)


In [35]:
bert2bert.to('cuda')

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [36]:
!nvidia-smi

Sat Jul 16 08:38:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    26W /  70W |   1858MiB / 15109MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [37]:
bert2bert

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [38]:
# bert2bert.save_pretrained("bert2bert")

In [39]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

In [40]:
batch_size = 32

In [41]:
bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id = tokenizer.sep_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id
bert2bert.config.vocab_size = bert2bert.config.encoder.vocab_size

In [42]:
bert2bert.config.max_length = 120
bert2bert.config.min_length = 120
# bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping = True
# bert2bert.config.length_penalty = 2.0
# bert2bert.config.num_beams = 4

In [43]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, 
    output_dir="./",
    logging_steps=2,
    save_steps=10,
    eval_steps=4,
    num_train_epochs=3,
    # logging_steps=1000,
    # save_steps=500,
    # eval_steps=7500,
    # warmup_steps=2000,
    # save_total_limit=3,
)

In [44]:
%%capture
!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu
!pip install datasets==1.0.2

In [45]:
import datasets

In [46]:
rouge = datasets.load_metric("rouge") # metric

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

In [47]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [48]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=training_set,#train_data,
    eval_dataset=validation_set#val_data,
)
trainer.train()

Using cuda_amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 940
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 90
The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces 

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
4,8.343800,7.661672,0.000000,0.000000,0.000000
8,7.022100,7.028683,0.000000,0.000000,0.000000
12,6.614200,6.746613,0.000000,0.000000,0.000000
16,6.432300,6.565997,0.000000,0.000000,0.000000
20,6.195600,6.582057,0.000000,0.000000,0.000000
24,6.162400,6.466913,0.000000,0.000000,0.000000
28,6.168200,6.397739,0.000000,0.000000,0.000000
32,6.146100,6.327081,0.000000,0.000000,0.000000
36,5.977000,6.299652,0.000000,0.000000,0.000000
40,5.860300,6.403637,0.000000,0.000000,0.000000


***** Running Evaluation *****
  Num examples = 50
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10/config.json
Model weights saved in ./checkpoint-10/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models 

TrainOutput(global_step=90, training_loss=6.155472670661078, metrics={'train_runtime': 281.8506, 'train_samples_per_second': 10.005, 'train_steps_per_second': 0.319, 'total_flos': 405463911667200.0, 'train_loss': 6.155472670661078, 'epoch': 3.0})

#### Evaluation

In [49]:
def generate_summary(batch):
    # cut off at BERT max length 512
    inputs = tokenizer(batch["SYSTEM_TRANSCRIPTION"].to_list(), padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = bert2bert.generate(input_ids, attention_mask=attention_mask)

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_summary"] = output_str

    return batch

In [50]:
batch = generate_summary(test_data)

In [51]:
batch

,index,HUMAN_TRANSCRIPTION,SYSTEM_TRANSCRIPTION,input_ids,attention_mask,decoder_input_ids,decoder_attention_mask,labels,pred_summary
0,50,σχυνοντες οποιος ην πτωχος ιωαννης μη,εχινοντεσ όποιος ηνπτωχος ιωαλνης ἐμη,"[101, 4569, 2658, 1907, 2781, 549, 4755, 1423,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 6301, 281, 11836, 605, 549, 239, 273, 18...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[101, 6301, 281, 11836, 605, 549, 239, 273, 18...",##ννςνννςνννννννννννννν
1,51,οικον εχων μη οικετην μη βουν αροτηρα μη,οικον χων όμε οικετην ύβηβον αροτηραταει,"[101, 10302, 7513, 273, 247, 438, 5850, 6338, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 10302, 576, 273, 452, 5850, 6338, 273, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 10302, 576, 273, 452, 5850, 6338, 273, 4...",##νννννννννννννννννννννννννννννννν
2,52,γηδιον μη κλινην μη τραπεζαν μη αρτον,χηδιον ψιξλένην λη τραυπεδαν μκαρτόν,"[101, 255, 267, 2927, 273, 256, 269, 274, 6999...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1244, 2927, 273, 452, 12341, 2389, 452, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[101, 1244, 2927, 273, 452, 12341, 2389, 452, ...",##νν
3,53,οποιος ηλιας οποιος των αγιων εκαστος,εποιος ψγίας εποιος των αγιων εκασος,"[101, 3830, 2251, 256, 9847, 3830, 2251, 358, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[101, 549, 5966, 549, 358, 4141, 12856, 278, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[101, 549, 5966, 549, 358, 4141, 12856, 278, 1...","##ςννννςνςνννςςς.ςνννννννςς.ν,ς"
4,54,οι περιηλθον εν μηλωταις εν αιγειοις δερμα,οιπεριἡλθον εν μηλωταις εναιγίοι σδερα,"[101, 363, 8024, 31108, 437, 452, 1480, 497, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 363, 518, 31108, 437, 452, 1480, 497, 27...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 363, 518, 31108, 437, 452, 1480, 497, 27...",##ννννννννννννν οννννννννννννννννν ονννν ονννν...
5,55,σιν υστερουμενοι θλιβομενοι κακουχουμενοι,σιο ὑπτερ ουμενοι θυκομεν οικακουχουεν,"[101, 1580, 275, 995, 1645, 247, 5094, 2313, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1580, 273, 15342, 5094, 2313, 22276, 567...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[101, 1580, 273, 15342, 5094, 2313, 22276, 567...",##ν.ςνννννννννν
6,56,ανθ ων υψωθησαν αι θυγατερες σιων και ε,ζθῶν ζυθης ακ ἀθυτα τερες εία ἡ και ε,"[101, 238, 6019, 238, 11883, 415, 3162, 8618, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 5148, 257, 273, 6485, 7641, 546, 1090, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 5148, 257, 273, 6485, 7641, 546, 1090, 2...",##ννςςςςςνννςνς
7,57,πορευθησαν υψηλω τραχηλω και εν νευμα,πορεύθηξακτοψηλωτρ χήλω και εμν εκμα,"[101, 12868, 902, 1698, 5299, 3414, 1480, 6847...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 12868, 7683, 13235, 1480, 13326, 3714, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[101, 12868, 7683, 13235, 1480, 13326, 3714, 2...",##νν καιν τον και και και και και και και. και...
8,58,σιν οφθαλμων και τη πορεια των ποδων α,ςίκε ὁ φθάλνα καὶ τὴ πορειατωη ποδυων α,"[101, 250, 2814, 265, 247, 10580, 271, 521, 34...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1580, 273, 33284, 344, 365, 903, 358, 20...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[101, 1580, 273, 33284, 344, 365, 903, 358, 20...",καιν..ν και και καιν καινς καινννς
9,59,μα συρουσαι τους χιτωνας και τοις ποσιν αμα,μαςύρογκαρ τον εχιτω ας κα τοις πος ιη αμα,"[101, 371, 6509, 12720, 362, 4569, 3001, 285, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 656, 10468, 2705, 364, 31343, 278, 344, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[101, 656, 10468, 2705, 364, 31343, 278, 344, ...",##νν.νννννννννννννννννννννν.ςς.ς.νν καιννν μεν...


In [52]:
batch_size = 16  # change to 64 for full evaluation

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882 in run_code  │
│                                                                                           │
│   2879 │   │   │   try:                                                                   │
│   2880 │   │   │   │   self.hooks.pre_run_code_hook()                                     │
│   2881 │   │   │   │   #rprint('Running code', repr(code_obj)) # dbg                      │
│ ❱ 2882 │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                  │
│   2883 │   │   │   finally:                                                               │
│   2884 │   │   │   │   # Reset our crash handler in place                                 │
│   2885 │   │   │   │   sys.excepthook = old_excepthook                                    │
│ <ipython-input-52-1b60aa113372>:3 in <module>                                             │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5487 in __getattr__         │
│                                                                                           │
│    5484 │   │   │   and self._info_axis._can_hold_identifiers_and_holds_name(name)        │
│    5485 │   │   ):                                                                        │
│    5486 │   │   │   return self[name]                                                     │
│ ❱  5487 │   │   return object.__getattribute__(self, name)                                │
│    5488 │                                                                                 │
│    5489 │   def __setattr__(self, name: str, value) -> None:                              │
│    5490 │   │   """                                                                       │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'DataFrame' object has no attribute 'map'

In [ ]:
rouge.compute(predictions=results["pred_summary"], references=results["highlights"], rouge_types=["rouge2"])["rouge2"].mid

---

In [ ]:
# # create tokenizer...
# tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

# input_ids = tokenizer(
#     "This is a long article to summarize", add_special_tokens=False, return_tensors="pt").input_ids
# labels = tokenizer("This is a short summary", return_tensors="pt").input_ids

# # train...
# loss = bert2bert(input_ids=input_ids, decoder_input_ids=labels, labels=labels).loss
# loss.backward()

In [ ]:
# from transformers import BertTokenizer

# # Load the BERT tokenizer.
# print('Loading BERT tokenizer...')
# # tokenizer = BertTokenizer.from_pretrained('pranaydeeps/Ancient-Greek-BERT', do_lower_case=True)
# tokenizer = BertTokenizer.from_pretrained('pranaydeeps/Ancient-Greek-BERT')

In [ ]:
# sentences = train_df.SYSTEM_TRANSCRIPTION
# sentences[0]

In [ ]:
# # Print the original sentence.
# print(' Original: ', sentences[0])

# # Print the sentence split into tokens.
# print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# # Print the sentence mapped to token ids.
# print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

#### Tokenize the dataset

In [ ]:
# # Tokenize all of the sentences and map the tokens to thier word IDs.
# input_ids = []
# attention_masks = []

# # For every sentence...
# for sent in sentences:
#     # `encode_plus` will:
#     #   (1) Tokenize the sentence.
#     #   (2) Prepend the `[CLS]` token to the start.
#     #   (3) Append the `[SEP]` token to the end.
#     #   (4) Map tokens to their IDs.
#     #   (5) Pad or truncate the sentence to `max_length`
#     #   (6) Create attention masks for [PAD] tokens.
#     encoded_dict = tokenizer.encode_plus(
#                         sent,                      # Sentence to encode.
#                         add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                         max_length = 64,           # Pad & truncate all sentences.
#                         pad_to_max_length = True,
#                         return_attention_mask = True,   # Construct attn. masks.
#                         return_tensors = 'pt',     # Return pytorch tensors.
#                    )
    
#     # Add the encoded sentence to the list.    
#     input_ids.append(encoded_dict['input_ids'])
    
#     # And its attention mask (simply differentiates padding from non-padding).
#     attention_masks.append(encoded_dict['attention_mask'])

# # Convert the lists into tensors.
# input_ids = torch.cat(input_ids, dim=0)
# attention_masks = torch.cat(attention_masks, dim=0)
# labels = torch.tensor(labels)

# # Print sentence 0, now as a list of IDs.
# print('Original: ', sentences[0])
# print('Token IDs:', input_ids[0])

#### Inference

In [ ]:
# # computing the B3 
# test_df["B3"] = test_df.SYSTEM_TRANSCRIPTION.apply(lmr)
# # computing the B3 CER
# test_df["B3_CER"] = test_df.apply(lambda row: pywer.cer([row.HUMAN_TRANSCRIPTION], [row.B3]), axis=1)
# print("B3's CER:", test_df.B3_CER.mean())
# # computing B3's CERR
# print((test_df.CER - test_df.B3_CER).mean())

In [ ]:
# !nvidia-smi

In [ ]:
# !pip install torchinfo

In [ ]:
# from torchinfo import summary

In [ ]:
# summary(bert2bert)

# Generating the Prediction File

In [ ]:
# using the 1st baseline (B6)
submission = pd.DataFrame(zip(test_df.IMAGE_PATH, test_df.B6), columns=["ImageID", "Transcriptions"])
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)

In [ ]:
# %aicrowd submission create -c htrec-2022 -f submission.csv